<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.1
    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-07-08 12:16:55
-------------------
qualified stocks: 96
with latest results: 31
still star stocks: 20
-------------------
Initial Investment:  1.17 C
CY Investment:  1.44 C
Reserve:  9.00 K
Current:  1.36 C
-------------------
Today PnL: -48.66 K (-0.36%)
Current PnL: -13.62 L (-9.49%)
CY Booked + Current PnL: -6.66 L (-4.64%)
-------------------
Total profit:  4.62 L
Total loss:  -18.24 L
-------------------
Total Booked + Current PnL: 20.32 L (17.43%)
Total Booked PnL: 33.94 L (29.12%)
Curr Year Booked PnL: 6.96 L (5.1%)
Prev Year Booked PnL: 26.98 L (23.15%)
Est FTT:  2.09 C
Est FTT PnL: 72.13 L (52.84%)
Deployed:  1.17 C
Current:  1.36 C
CAGR/XIRR %: 10.65%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
4,APOLLOHOSP,6904.43,8285.00,-16.41,H-LC,91.67,113925.0,10359.0,10344.0,-0.28,10.00,9.08,20.00,32.0,1.00,0.84,25.48,X40N,BTT,HEALTHCARE
39,HINDALCO,651.95,761.55,-19.90,H-LC,85.42,110096.0,5784.0,11747.0,-0.58,5.54,10.67,16.81,27.0,0.49,0.81,22.48,X5K,ATH,METALS
92,VIPIND,488.80,489.00,-211.87,H-SC,98.96,80681.0,-14635.0,14676.0,-0.25,-15.35,18.19,0.04,157.0,-1.00,0.59,59.13,OX40N,NTT,MISC
71,SBIN,760.30,863.00,-11.51,M-LC,32.29,214014.0,12534.0,14681.0,0.09,6.22,6.86,13.51,77.0,0.85,1.57,19.60,XY25,NTT,BANKS
88,UNIONBANK,123.87,163.00,-2.93,M-LC,20.83,170209.0,29369.0,15115.0,-1.62,20.85,8.88,31.59,88.0,1.94,1.25,47.85,XY24,NTT,BANKS


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BANKINDIA,116.91,190.00,-32.52,M-MC,7.29,178931.0,-877.0,113281.0,-1.17,-0.49,63.31,62.52,173.0,-0.01,1.31,33.43,XR,NTT,BANKS
38,HEROMOTOCO,4311.81,6039.03,-6.67,H-MC,35.42,150360.0,-553.0,61001.0,-0.26,-0.37,40.57,40.06,103.0,-0.01,1.10,22.99,AR,ATH,AUTO
60,NATIONALUM,189.63,247.44,-50.15,H-MC,57.29,99974.0,-530.0,31172.0,0.04,-0.53,31.18,30.49,97.0,-0.02,0.73,31.71,MH,ATH,METALS


In [11]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
16,BLUSPRING,226.45,84.77,NaN,NaN,50.00,18565.0,-31028.0,0.0,3.80,-62.56,0.00,-62.57,270.0,-inf,0.14,24.50,XY24,ATH,MISC
85,TRIDENT,37.35,48.00,25.04,M-SC,72.92,78694.0,-13560.0,39866.0,2.38,-14.70,50.66,28.51,233.0,-0.34,0.58,34.66,XR,NTT,TEXTILES
90,VALIANTORG,512.64,838.00,-360.76,H-SC,26.04,110760.0,-22526.0,107116.0,2.13,-16.90,96.71,63.47,151.0,-0.21,0.81,85.66,XR,NTT,CHEMICALS
32,ENRIN,1377.95,3063.70,NaN,NaN,95.83,76592.0,42143.0,0.0,2.12,122.34,0.00,122.34,274.0,inf,0.56,17.12,AR,ATH,ELECTRICAL
8,AWL,325.39,485.00,-69.07,H-MC,25.00,214376.0,-54071.0,185757.0,1.03,-20.14,86.65,49.05,98.0,-0.29,1.57,10.22,XY24,NTT,FMCG


In [12]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
24,COFFEEDAY,59.14,80.00,-57.75,L-SC,45.83,63360.0,-50189.0,90237.0,-3.17,-44.20,142.42,35.27,265.0,-0.56,0.46,55.08,XR,NTT,HOTELS
65,RELAXO,902.64,1176.00,-30.38,H-SC,93.75,62957.0,-52581.0,87573.0,-2.86,-45.51,139.10,30.28,131.0,-0.60,0.46,22.99,X40N,NTT,FOOTWEAR
3,ALKYLAMINE,2347.98,4546.37,7.94,X-SC,97.92,100500.0,-463.0,94993.0,-2.54,-0.46,94.52,93.63,67.0,-0.00,0.74,49.69,SR,ATH,CHEMICALS
0,5PAISA,564.75,565.00,153.61,H-SC,28.12,136653.0,-50279.0,50357.0,-2.52,-26.90,36.85,0.04,149.0,-1.00,1.00,30.73,OX40N,NTT,FINANCE
19,CAMPUS,294.86,393.00,-27.00,M-SC,4.17,151951.0,-15824.0,71660.0,-2.13,-9.43,47.16,33.28,206.0,-0.22,1.11,17.64,XY24,NTT,FOOTWEAR


In [13]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
87,UJJIVANSFB,52.77,53.0,47.02,M-SC,23.96,127197.0,-15282.0,15900.0,0.23,-10.73,12.50,0.44,247.0,-0.96,0.93,50.13,OX40N,NTT,BANKS
79,SYMPHONY,1306.42,1306.0,-29.18,M-SC,29.17,150899.0,-20242.0,20190.0,0.03,-11.83,13.38,-0.03,191.0,-1.00,1.11,8.45,OX40N,NTT,DURABLES
46,INDIGOPNTS,1407.73,1408.0,127.12,H-SC,94.79,148887.0,-25672.0,25698.0,-0.01,-14.71,17.26,0.02,133.0,-1.00,1.09,29.11,OX40N,NTT,PAINTS
92,VIPIND,488.80,489.0,-211.87,H-SC,98.96,80681.0,-14635.0,14676.0,-0.25,-15.35,18.19,0.04,157.0,-1.00,0.59,59.13,OX40N,NTT,MISC
81,TATAELXSI,7332.28,7332.0,-18.66,X-MC,6.25,85449.0,-17203.0,17201.0,-1.16,-16.76,20.13,-0.00,59.0,-1.00,0.63,29.89,OX40N,NTT,IT


In [14]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,TCS,3794.03,4998.00,-18.38,X-LC,43.75,265972.0,-29962.0,123863.0,-0.05,-10.12,46.57,31.73,1.0,-0.24,1.95,6.46,X40,BTT,IT
48,INFY,1461.46,2275.00,-8.83,X-LC,79.17,292486.0,30885.0,114742.0,0.43,11.81,39.23,55.67,2.0,0.27,2.14,18.63,X40,BTT,IT
40,HINDUNILVR,2413.81,2723.00,-13.95,X-LC,62.50,269640.0,-707.0,35323.0,-0.12,-0.26,13.10,12.81,5.0,-0.02,1.98,12.77,XY25,NTT,FMCG
17,BRITANNIA,4983.33,6446.05,17.22,X-LC,69.79,268962.0,39729.0,27569.0,-0.63,17.33,10.25,29.35,10.0,1.44,1.97,27.80,XY25,ATH,FMCG
74,SIEMENS,4406.90,7969.85,-2.86,X-LC,27.08,82100.0,-28072.0,117148.0,0.18,-25.48,142.69,80.85,11.0,-0.24,0.60,20.92,AR,ATH,ELECTRICAL


In [15]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
32,ENRIN,1377.95,3063.70,NaN,NaN,95.83,76592.0,42143.0,0.0,2.12,122.34,0.00,122.34,274.0,inf,0.56,17.12,AR,ATH,ELECTRICAL
28,DIGITIDE,188.38,237.72,NaN,NaN,33.33,50300.0,9045.0,1760.0,0.52,21.92,3.50,26.19,272.0,5.14,0.37,18.42,XY24,ATH,IT
52,JPPOWER,18.73,26.20,-13.94,L-SC,100.00,173270.0,31428.0,25141.0,1.02,22.16,14.51,39.88,262.0,1.25,1.27,75.59,XY24,NTT,POWER
88,UNIONBANK,123.87,163.00,-2.93,M-LC,20.83,170209.0,29369.0,15115.0,-1.62,20.85,8.88,31.59,88.0,1.94,1.25,47.85,XY24,NTT,BANKS


In [16]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
68,SAIL,130.64,228.00,92.75,M-MC,52.08,231075.0,6113.0,161545.0,0.54,2.72,69.91,74.53,189.0,0.04,1.69,34.16,XY24,BTT,STEEL
54,KPIGREEN,497.21,731.64,24.30,H-SC,30.21,127197.0,1900.0,57175.0,-1.60,1.52,44.95,47.15,143.0,0.03,0.93,59.13,MH,ATH,POWER
20,CAMS,3643.00,5250.99,2.09,H-SC,18.75,114682.0,12678.0,32340.0,-1.12,12.43,28.20,44.14,126.0,0.39,0.84,30.69,X40N,ATH,MISC
95,WIPRO,243.46,420.00,-2.86,M-LC,89.58,166501.0,15556.0,93907.0,0.30,10.31,56.40,72.51,71.0,0.17,1.22,14.69,XR,NTT,IT
88,UNIONBANK,123.87,163.00,-2.93,M-LC,20.83,170209.0,29369.0,15115.0,-1.62,20.85,8.88,31.59,88.0,1.94,1.25,47.85,XY24,NTT,BANKS


In [17]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
32,ENRIN,1377.95,3063.70,NaN,NaN,95.83,76592.0,42143.0,0.0,2.12,122.34,0.00,122.34,274.0,inf,0.56,17.12,AR,ATH,ELECTRICAL
94,WHIRLPOOL,1167.49,2270.00,-39.69,M-SC,46.88,198759.0,27138.0,134937.0,-1.51,15.81,67.89,94.43,201.0,0.20,1.46,46.70,XR,NTT,DURABLES
95,WIPRO,243.46,420.00,-2.86,M-LC,89.58,166501.0,15556.0,93907.0,0.30,10.31,56.40,72.51,71.0,0.17,1.22,14.69,XR,NTT,IT
45,INDIAMART,2327.09,4911.36,-48.26,H-SC,71.88,134874.0,11538.0,125433.0,0.25,9.36,93.00,111.05,123.0,0.09,0.99,32.15,AR,ATH,MISC
54,KPIGREEN,497.21,731.64,24.30,H-SC,30.21,127197.0,1900.0,57175.0,-1.60,1.52,44.95,47.15,143.0,0.03,0.93,59.13,MH,ATH,POWER


In [18]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
16,BLUSPRING,226.45,84.77,NaN,NaN,50.00,18565.0,-31028.0,0.0,3.80,-62.56,0.00,-62.57,270.0,-inf,0.14,24.50,XY24,ATH,MISC
24,COFFEEDAY,59.14,80.00,-57.75,L-SC,45.83,63360.0,-50189.0,90237.0,-3.17,-44.20,142.42,35.27,265.0,-0.56,0.46,55.08,XR,NTT,HOTELS
32,ENRIN,1377.95,3063.70,NaN,NaN,95.83,76592.0,42143.0,0.0,2.12,122.34,0.00,122.34,274.0,inf,0.56,17.12,AR,ATH,ELECTRICAL
58,MASFIN,326.60,399.50,-18.25,H-SC,70.83,93105.0,-4875.0,26749.0,0.39,-4.98,28.73,22.32,145.0,-0.18,0.68,35.03,XR,ATH,FINANCE
9,BAJAJFINSV,1994.80,2758.00,-1.53,X-LC,41.67,98343.0,598.0,36800.0,0.03,0.61,37.42,38.26,18.0,0.02,0.72,31.31,X40,BTT,FINANCE


In [19]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
52,JPPOWER,18.73,26.20,-13.94,L-SC,100.00,173270.0,31428.0,25141.0,1.02,22.16,14.51,39.88,262.0,1.25,1.27,75.59,XY24,NTT,POWER
92,VIPIND,488.80,489.00,-211.87,H-SC,98.96,80681.0,-14635.0,14676.0,-0.25,-15.35,18.19,0.04,157.0,-1.00,0.59,59.13,OX40N,NTT,MISC
3,ALKYLAMINE,2347.98,4546.37,7.94,X-SC,97.92,100500.0,-463.0,94993.0,-2.54,-0.46,94.52,93.63,67.0,-0.00,0.74,49.69,SR,ATH,CHEMICALS
15,BLUESTARCO,1646.70,2337.55,-3.59,H-SC,96.88,180790.0,16120.0,52971.0,-1.03,9.79,29.30,41.95,124.0,0.30,1.33,17.99,X40N,ATH,AC
32,ENRIN,1377.95,3063.70,NaN,NaN,95.83,76592.0,42143.0,0.0,2.12,122.34,0.00,122.34,274.0,inf,0.56,17.12,AR,ATH,ELECTRICAL


In [20]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,18.95
1,25,41.94
2,50,70.61


In [21]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.38
LC    34.01
MC    21.54
Name: CurrAlloc%, dtype: float64

In [22]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.92
X40      12.96
XY25     12.06
X40N     11.92
XR       10.02
AR        8.19
OX40N     8.18
MH        1.66
X5K       1.45
X200      1.42
SR        1.22
Name: CurrAlloc%, dtype: float64

In [23]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    20.41
M-SC    15.88
H-LC    14.15
X-LC    12.94
H-MC     9.53
X-MC     7.60
M-LC     5.88
X-SC     4.45
M-MC     4.05
L-SC     2.64
L-LC     1.04
L-MC     0.36
Name: CurrAlloc%, dtype: float64

In [24]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.24,-0.63,34.35
IT,12.14,-9.83,63.76
FINANCE,8.47,-12.82,53.76
BANKS,7.69,-8.63,52.95
MISC,5.83,-20.87,77.52
PAINTS,5.70,-12.40,34.55
ELECTRICAL,5.48,1.48,44.83
HEALTHCARE,4.65,-1.79,31.53
AUTO,4.32,-14.77,65.23


In [25]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2672972.0,26
XR,1090859.0,13
AR,1031712.0,10
X40,687238.0,10
X40N,538475.0,11
XY25,474229.0,8
OX40N,313905.0,11
SR,177609.0,2
MH,88347.0,2


In [26]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,1998581.0,20
M-SC,1778218.0,21
H-LC,612302.0,12
X-LC,544319.0,8
H-MC,533773.0,8
X-MC,416977.0,6
M-MC,416890.0,3
X-SC,312880.0,5
M-LC,259616.0,4


In [27]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      895370.0      6
M-SC       XY24      813873.0      7
H-SC       AR        529621.0      3
M-SC       XR        338217.0      4
M-MC       XY24      303609.0      2
H-MC       XY24      300835.0      3
X-LC       X40       275405.0      3
H-SC       XR        215176.0      3
           X40N      210508.0      4
H-LC       X40N      197241.0      4
M-SC       AR        184776.0      2
           OX40N     164630.0      5
X-MC       X40       157187.0      2
H-LC       X40       154274.0      3
L-SC       XR        152439.0      2
M-LC       XY24      151028.0      2
X-SC       XR        123960.0      1
M-SC       XY25      122089.0      1
X-LC       AR        117148.0      1
M-MC       XR        113281.0      1
X-MC       XY25      103820.0      1
X-LC       XY25      102674.0      3
X-SC       SR         94993.0      1
M-LC       XR         93907.0      1
H-SC       OX40N      90731.0      3
X-MC       XY24       86461.0      1
H-MC       XY25       83084.0      1
M-SC       SR         82616.0      1
H-LC       AR         80832.0      1
M-SC       X40        72017.0      1
H-LC       X5K        70632.0      2
           X200       67012.0      1
H-MC       AR         61001.0      1
L-SC       AR         58334.0      1
H-SC       MH         57175.0      1
L-MC       XR         53879.0      1
X-SC       XY24       52584.0      1
X-MC       X40N       52308.0      1
X-LC       X40N       49092.0      1
L-LC       XY25       47881.0      1
H-LC       XY24       42311.0      1
X-SC       OX40N      41343.0      2
H-MC       MH         31172.0      1
           X40N       29326.0      1
           X40        28355.0      1
L-SC       XY24       25141.0      1
X-MC       OX40N      17201.0      1
M-LC       XY25       14681.0      1

In [28]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 27.0 seconds
